In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [100]:
#A site url for the resource
url = 'https://ors.aqrb.go.tz/firms_list'

In [101]:
#Instantiate a parser object
hp = HTMLTableParser()

In [91]:
#Retrieve the table from a list
table = hp.parse_url(url)[1]

In [92]:
table.head()

,0,1,2,3,4,5,6,7
0,1.0,3D Arch. Consult,Architectural firm,Local,Box 3000 Morogoro,,jeje@yahoo.com,0713760088
1,2.0,A + P Consultants Ltd,Architectural firm,Local,Box 2076 DSM,,apggcl@yahoo.com,0754781150
2,3.0,A. V. Consult,Architectural firm,Local,Box 33213 DSM,,avconsult2005@yahoo.com,0741461474
3,4.0,Abacus Consults,Architectural firm,Local,Box 2453 Mwanza,,de_rwabuhanga@rocketmail.com,0782254185
4,5.0,Abyss Creations,Architectural firm,Local,Box 76615 DSM,,ajmcha2000@yahoo.com,0718853037


In [102]:
#Assign column names
table.columns = ['S/N','Name','Type','Category','Contacts','City','Email','Phone number']

In [103]:
table.head()

,S/N,Name,Type,Category,Contacts,City,Email,Phone number
0,1.0,3D Arch. Consult,Architectural firm,Local,Box 3000 Morogoro,,jeje@yahoo.com,0713760088
1,2.0,A + P Consultants Ltd,Architectural firm,Local,Box 2076 DSM,,apggcl@yahoo.com,0754781150
2,3.0,A. V. Consult,Architectural firm,Local,Box 33213 DSM,,avconsult2005@yahoo.com,0741461474
3,4.0,Abacus Consults,Architectural firm,Local,Box 2453 Mwanza,,de_rwabuhanga@rocketmail.com,0782254185
4,5.0,Abyss Creations,Architectural firm,Local,Box 76615 DSM,,ajmcha2000@yahoo.com,0718853037


In [99]:
table.shape

(336, 8)

In [104]:
#Export the table to excel
table.to_excel('Architetcs_QS_firms.xlsx')